In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import climetlab as cml
from climetlab_cems_flood.adapter import Adapter
import climetlab_cems_flood as cmf

In [3]:
nwse = [50.972204,5.450796, 46.296530, 11.871059] # Ryne

In [4]:
class MyMerger():
    def __init__(self, *args, **kwargs):
        pass
    def merge(self, paths, **kwargs):
        return xr.open_mfdataset(paths)


In [5]:
forecast1 = cml.load_dataset(
            'cems-flood-glofas-forecast',
            model='lisflood',
            product_type='ensemble_perturbed_forecasts',
            system_version='operational',
            period= '20210710-20210711',#'2001-200401-04*',
            leadtime_hour = '24-120',
            variable="river_discharge_in_the_last_24_hours",
            #area= nwse,
            split_on = ['day'],
            threads = 6,
            merger=None
        )

By downloading data from this dataset, you agree to the terms and conditions defined at https://github.com/ecmwf-lab/climetlab_cems_flood/LICENSEIf you do not agree with such terms, do not download the data. 


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
ds = forecast1.to_xarray()

In [8]:
ds

<xarray.Dataset>
Dimensions:                  (realization: 50, forecast_reference_time: 2,
                              leadtime: 5, lat: 1500, lon: 3600)
Coordinates:
  * realization              (realization) int64 1 2 3 4 5 6 ... 46 47 48 49 50
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2021-07...
  * leadtime                 (leadtime) timedelta64[ns] 1 days 2 days ... 5 days
  * lat                      (lat) float64 -59.95 -59.85 -59.75 ... 89.85 89.95
  * lon                      (lon) float64 -179.9 -179.8 -179.8 ... 179.8 179.9
    time                     (forecast_reference_time, leadtime) datetime64[ns] ...
Data variables:
    dis24                    (realization, forecast_reference_time, leadtime, lat, lon) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-05-23T14:11 GRIB to CDM+CF via cfgrib-0.9.1...

In [14]:
ds = ds.chunk({"realization":1,
                      "forecast_reference_time": 1,
                      "leadtime": -1,
                      "lat": 250,
                      "lon": 250})

In [ ]:
ds.to_zarr("/data/temp/clim_zarr_big.zarr", mode= "w")

In [1]:
def ps(x):
    print(x)

In [3]:
a =lambda: ps(x=10)

In [5]:
a()

10


In [10]:
ada = Adapter(forecast1, 
              output_folder= '/data/temp', 
              output_name_prefix= 'forecast_bg')

In [12]:
ada.to_zarr(True, 
            {"dis24":{"realization":1,
                      "forecast_reference_time": 1,
                      "leadtime": -1,
                      "lat": 250,
                      "lon": 250},
            "forecast_reference_time":None,
            "lat":None,
            "lon":None,
            "leadtime":None},
            '25M', 
            'temp.zarr')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/iacopo/.pyenv/versions/riverv-dev/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_124171/1027486510.py", line 1, in <cell line: 1>
    ada.to_zarr(True,
  File "/home/iacopo/dev/climetlab-cems-flood/climetlab_cems_flood/adapter.py", line 74, in to_zarr
    dsnc.to_zarr(out_name_fmt, mode ="w")
  File "/home/iacopo/.pyenv/versions/riverv-dev/lib/python3.10/site-packages/xarray/core/dataset.py", line 2036, in to_zarr
    return to_zarr(
  File "/home/iacopo/.pyenv/versions/riverv-dev/lib/python3.10/site-packages/xarray/backends/api.py", line 1432, in to_zarr
    writes = writer.sync(compute=compute)
  File "/home/iacopo/.pyenv/versions/riverv-dev/lib/python3.10/site-packages/xarray/backends/common.py", line 166, in sync
    delayed_store = da.store(
  File "/home/iacopo/.pyenv/versions/riverv-dev/lib/python3.10/site-packages/

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,10))
for i,c in enumerate(["black","grey","blue","red","green","orange"]):
    _ = plt.plot(ds.dis24.isel(forecast_reference_time=i).time.values,
                 ds.dis24.isel(forecast_reference_time=i,drop=True).mean(["lat","lon"]).values.T,
                 color=c)

In [ ]:
forecast = cml.load_dataset(
            'cems-flood-glofas-forecast',
            model='lisflood',
            product_type='ensemble_perturbed_forecasts',
            system_version='operational',
            period= '20210710-20210715',#'2001-200401-04*',
            leadtime_hour = '24-720',
            variable="river_discharge_in_the_last_24_hours",
            area= nwse, # list of ROI
            split_on = [('day',1)],
            threads = 5
        )

In [15]:
import zarr

In [16]:
z = zarr.open_group("/data/temp/cems-flood-glofas-forecast-forecast.zarr/")

In [24]:
z.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='dis24 (50, 6, 5, 47, …